In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

In [ ]:
%cd /gdrive/MyDrive/Final project/skindata/Training/02.라벨링데이터

!unzip -qq "/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Training/02.라벨링데이터/TL.zip"

In [ ]:
%cd /gdrive/MyDrive/Final project/skindata/Training/01.원천데이터

!unzip -qq "/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Training/01.원천데이터/TS.zip"

In [ ]:
%cd /gdrive/MyDrive/Final project/skindata/Validation/01.원천데이터

!unzip -qq "/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Validation/01.원천데이터/VS.zip"

In [ ]:
%cd /gdrive/MyDrive/Final project/skindata/Validation/02.라벨링데이터

!unzip -qq "/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터/Validation/02.라벨링데이터/VL.zip"

Training/01.원천데이터, 02.라벨링데이터<br>
Validation/01.원천데이터, 02.라벨링데이터만들고 사용




# 데이터 확인

#### JPG, JSON 파일 불러오기

In [ ]:
import os
from PIL import Image
import json

# 데이터 폴더 경로
data_dir = '/gdrive/MyDrive/Final project/skindata/Validation'
raw_data_dir = os.path.join(data_dir, '01.원천데이터')
label_data_dir = os.path.join(data_dir, '02.라벨링데이터')

# 파일 리스트를 저장할 딕셔너리 초기화
images_dict = {}
json_dict = {}

# 원천 데이터 폴더 내의 모든 jpg 파일 탐색
for root, dirs, files in os.walk(raw_data_dir):
    for file in files:
        if file.endswith('.jpg'):
            file_path = os.path.join(root, file)
            file_name = os.path.splitext(file)[0]  # 확장자를 제외한 파일 이름
            images_dict[file_name] = Image.open(file_path)

# 라벨링 데이터 폴더 내의 모든 json 파일 탐색
for root, dirs, files in os.walk(label_data_dir):
    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(root, file)
            file_name = os.path.splitext(file)[0]  # 확장자를 제외한 파일 이름
            with open(file_path, 'r', encoding='utf-8') as f:
                json_dict[file_name] = json.load(f)

In [ ]:
len(images_dict), len(json_dict)

In [ ]:
images_dict['0001_01_F']

#### 이미지명으로 라벨링 데이터 불러오기
- 1개의 이미지 당 9개의 라벨링 데이터 존재
- 한 사람당 각도가 다르게 촬영된 이미지 파일이 있어 9의배수로 출력

In [ ]:
# 찾고자 하는 파일 이름
search_filename = '0001_01_Fb.jpg'

# filename으로 검색하여 항목 찾기
matching_entries = {key: value for key, value in json_dict.items() if value['info']['filename'] == search_filename}

# 결과를 하나의 딕셔너리로 묶기
combined_dict = {"total": matching_entries}

# 결과 출력
print(json.dumps(combined_dict, indent=4, ensure_ascii=False))

In [ ]:
combined_dict

In [ ]:
json_dict

In [ ]:
json_dict['0001_01_F_01']['images']['bbox']

# 모델학습 도전

#### YOLOv5 다운로드

In [ ]:
# YOLOv5 저장소를 클론하고 필요한 패키지를 설치
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

In [ ]:
import os
import json
import cv2
import numpy as np
from ultralytics import YOLO
from sklearn.model_selection import train_test_split

In [ ]:
def preprocess_data(data_dir, split):
    images = []
    labels = []

    split_dir = os.path.join(data_dir, split)
    img_dir = os.path.join(split_dir, '01.원천데이터')
    label_dir = os.path.join(split_dir, '02.라벨링데이터')

    for root, _, files in os.walk(img_dir):
        for file in files:
            if file.endswith('.jpg'):
                img_path = os.path.join(root, file)

                img = cv2.imread(img_path)
                height, width = img.shape[:2]

                base_name = file.split('.')[0]  # 예: '0001_01_F'

                # 해당 이미지에 대한 모든 JSON 파일 찾기
                json_files = [f for f in os.listdir(label_dir) if f.startswith(base_name) and f.endswith('.json')]

                for json_file in json_files:
                    json_path = os.path.join(label_dir, json_file)

                    with open(json_path, 'r', encoding='utf-8') as f:
                        label_data = json.load(f)

                    facepart = label_data['images']['facepart']
                    x, y, w, h = label_data['images']['bbox']

                    # YOLO 형식으로 변환
                    x_center = (x + w/2) / width
                    y_center = (y + h/2) / height
                    w = w / width
                    h = h / height

                    labels.append([img_path, facepart, x_center, y_center, w, h])

                images.append(img_path)

    return images, labels

In [ ]:
# 2. YOLO 모델 학습
def train_yolo_model(data_dir):
    # YOLO 모델 초기화
    model = YOLO('yolov8n.yaml')

    # 데이터셋 설정
    dataset = {
        'path': data_dir,
        'train': 'Training/01.원천데이터',
        'val': 'Validation/01.원천데이터',
        'nc': 9,  # 클래스 수 (얼굴 부위 9개)
        'names': ['전체', '이마', '미간', '좌측눈가', '우측눈가', '좌측볼', '우측볼', '입술', '턱']
    }

    # YAML 파일 생성
    with open('dataset.yaml', 'w') as f:
        json.dump(dataset, f)

    # 모델 학습
    results = model.train(data='dataset.yaml', epochs=20, imgsz=640)

    return model

In [ ]:
# 3. 예측 및 피부 상태 진단
def predict_skin_condition(model, image_path, label_dir):
    # 이미지 로드
    img = cv2.imread(image_path)

    # YOLO 예측
    results = model(img)

    # 각 감지된 부위에 대해 피부 상태 진단
    for r in results:
        boxes = r.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0]
            cls = int(box.cls[0])
            conf = float(box.conf[0])

            if conf > 0.5:  # 신뢰도가 50% 이상인 경우만 처리
                part_name = model.names[cls]

                # 해당 부위의 라벨 데이터 찾기
                json_path = os.path.join(label_dir, f"{os.path.basename(image_path).split('.')[0]}_{cls}.json")

                if os.path.exists(json_path):
                    with open(json_path, 'r', encoding='utf-8') as f:
                        label_data = json.load(f)

                    print(f"부위: {part_name}")

                    # 각 부위별 특성 출력
                    if 'annotations' in label_data:
                        for key, value in label_data['annotations'].items():
                            print(f"{key}: {value}")

                    if 'equipment' in label_data and label_data['equipment']:
                        for key, value in label_data['equipment'].items():
                            print(f"{key}: {value}")

                    print("---")

In [ ]:
# 메인 실행 코드
if __name__ == "__main__":
    data_dir = "/gdrive/MyDrive/Final project/skindata"
    model = train_yolo_model(data_dir)

    # 예측 검증
    test_image_path = "/gdrive/MyDrive/Final project/skindata/test/01.원천데이터/1091_03_F.jpg"
    test_label_dir = "/gdrive/MyDrive/Final project/skindata/test/02.라벨링데이터"
    predict_skin_condition(model, test_image_path, test_label_dir)